In [42]:
import pandas as pd
import ee
import os

In [43]:
path = os.getcwd()
home_path = os.path.dirname(path)
gis_folder = f'{home_path}\\GIS'
data_folder = f'{home_path}\\Data'

In [44]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AVHEtk4COHIQ_HKP6Hf_TsLF2v6BUMdDntUclueKw9xhlFU5ixXRXKXDvMc



Successfully saved authorization token.


In [45]:
IMERG = ee.ImageCollection('NASA/GPM_L3/IMERG_V06')


In [46]:
# Initial date of interest (inclusive).
i_date = '2010-01-01'
# Final date of interest (exclusive).
f_date = '2015-01-01'

In [47]:
# Selection of appropriate bands and dates for precip
selected_dates = IMERG.filterDate(i_date, f_date)

precip_dates = selected_dates.select('precipitationCal')

In [48]:
scale = 1000 #m?
# Define the urban location of interest as a point near albany, NY
u_lon = -74
u_lat = 43
u_poi = ee.Geometry.Point(u_lon, u_lat)

In [49]:
lst_data = precip_dates.getRegion(u_poi, scale).getInfo()

In [50]:
df = pd.DataFrame(data=lst_data[1:],columns=lst_data[0])

In [51]:
df.index = df.apply(lambda x: pd.Timestamp(f"{x.id[:4]}-{x.id[4:6]}-{x.id[6:8]}"),axis=1)

In [52]:
df.rename(columns={"precipitationCal":"P"},inplace=True)
df[["P"]].to_parquet(f"{data_folder}\\P\\IMERG.parquet")

In [53]:
# repeat for MODIS evaporation data

In [54]:
MODIS = ee.ImageCollection("MODIS/006/MOD16A2")
selected_dates_MODIS = MODIS.filterDate(i_date, f_date)
ET_dates = selected_dates_MODIS.select('PET')
lst_data_modis = ET_dates.getRegion(u_poi, scale).getInfo()
df_modis = pd.DataFrame(data=lst_data_modis[1:],columns=lst_data_modis[0])
df_modis.index  = df_modis.apply(lambda x: pd.Timestamp(f"{x.id[:4]}-{x.id[5:7]}-{x.id[8:10]}"),axis=1)
df_modis[["PET"]].to_parquet(f"{data_folder}\\EP\\MODIS.parquet")

In [55]:
df_modis

,id,longitude,latitude,time,PET
2010-01-01,2010_01_01,-73.998722,42.997861,1262304000000,NaN
2010-01-09,2010_01_09,-73.998722,42.997861,1262995200000,NaN
2010-01-17,2010_01_17,-73.998722,42.997861,1263686400000,NaN
2010-01-25,2010_01_25,-73.998722,42.997861,1264377600000,NaN
2010-02-02,2010_02_02,-73.998722,42.997861,1265068800000,NaN
...,...,...,...,...,...
2014-11-25,2014_11_25,-73.998722,42.997861,1416873600000,NaN
2014-12-03,2014_12_03,-73.998722,42.997861,1417564800000,NaN
2014-12-11,2014_12_11,-73.998722,42.997861,1418256000000,NaN
2014-12-19,2014_12_19,-73.998722,42.997861,1418947200000,36.0


repeat for GRACE

In [73]:
GRACE = ee.ImageCollection("NASA/GRACE/MASS_GRIDS/MASCON_CRI")
selected_dates_GRACE = GRACE.filterDate(i_date, f_date)
GRACE_dates = selected_dates_GRACE.select('lwe_thickness')
lst_data_grace = GRACE_dates.getRegion(u_poi, scale).getInfo()
df_grace = pd.DataFrame(data=lst_data_grace[1:],columns=lst_data_grace[0])
df_grace.index  = df_grace.apply(lambda x: pd.Timestamp(f"{x.id[:4]}-{x.id[4:6]}-{x.id[6:8]}"),axis=1)
df_grace["graceJPL"] = df_grace["lwe_thickness"]
df_grace[["graceJPL"]].to_parquet(f"{data_folder}\\Grace\\GRACE.parquet")

In [74]:
GRACE_error = ee.ImageCollection("NASA/GRACE/MASS_GRIDS/MASCON_CRI")
selected_dates_GRACE_error = GRACE_error.filterDate(i_date, f_date)
GRACE_error_dates = selected_dates_GRACE_error.select('uncertainty')
lst_data_GRACE_error = GRACE_error_dates.getRegion(u_poi, scale).getInfo()
df_GRACE_error = pd.DataFrame(data=lst_data_GRACE_error[1:],columns=lst_data_GRACE_error[0])
df_GRACE_error.index  = df_GRACE_error.apply(lambda x: pd.Timestamp(f"{x.id[:4]}-{x.id[4:6]}-{x.id[6:8]}"),axis=1)
df_GRACE_error["graceJPLerror"] = df_GRACE_error["uncertainty"]
df_GRACE_error[["graceJPLerror"]].to_parquet(f"{data_folder}\\Grace\\GRACE_error.parquet")